<a href="https://colab.research.google.com/github/ANANTHUPADHYA/ML-Final-Exam/blob/main/FinalExam_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Team Elite:
* Priyanka Devendran 
* Ananth Upadya
* Deesha Desai


1. Clone this repo : https://github.com/aarsanjani/nlp-poets.git (Links to an external site.)

2. it has two notebooks, one you have used and seen before: nlp-summarization-pos.ipynb (Links to an external site.)

3. the new one is some background code snippets and references for your task 3, which is to Name Your Topic: name_your_topic_background.ipynb (Links to an external site.)

-------

tasks for final exam
1. NLP Data. Pick 2 poets, scrape 10 poems each . Put them in a json datastructure and save it under the poets name, in your team drive under folder 'finalexam'

2. CLAIM YOUR POETS!!! Note: your team has  to claim their 2 poets by entering in their names in this spreadsheet with a timestamp of when you entered this: no duplicate poets allowed !

https://docs.google.com/spreadsheets/d/1CQ-UknoxXxyCb_-Uv0f95NChKEOHZB81t5pa4Piff9A/edit?usp=sharing (Links to an external site.)

---for each poem of each poet:
3. Run POS and extract verbs, nouns and adjectives (use notebook given as an example)

4. Summarize each poem 

5. Store the summary, POS data in your json poet data structure and save to disk.

6. Interchange the verbs and adjectives of the two poets and 

6a. save the data structure for the new poems:  poet1-poet2.json (replace poet1 adjectives and verbs with poet2's ) and poet2-poet1.json 

6b. print out the new poems and save them to disk poet1-poet2.txt and poet2-poet1.txt [these files will contain 10 poems each)].
remember to measure and compare the cohesion and perplexity of the original poems and the new poems you generate !!

7. Combine poet1's and poet2's poem text and summarize it using the transformer

8. Run topic modeling on the combined text of each poet: what are the top 5 topics they are writing about?

9. Name the topics: use the new notebook : name_your_topic_background.ipynb (Links to an external site.) , to try and find the closest word that describes topic 0, topic 1, topic 2 : instead of topic 0, what is the closest word in that cluster that describes the topic?

 

Pls Submit your team colab link, actual .ipynb notebook in canvas, also pls enter the colab link in the spreadsheet:

https://docs.google.com/spreadsheets/d/1CQ-UknoxXxyCb_-Uv0f95NChKEOHZB81t5pa4Piff9A/edit?usp=sharing (Links to an external site.)

 

Godspeed NLP Poets!!

**Contribution**
* Ananth Upadya

1. NLP Data. Pick 2 poets, scrape 10 poems each . Put them in a json datastructure and save it under the poets name, in your team drive under folder 'finalexam'

2. Note: your team has  to claim their 2 poets by entering in their names in this spreadsheet with a timestamp of when you entered this: no duplicate poets allowed.

3. Run POS and extract verbs, nouns and adjectives (use notebook given as an example)

* Priyanka Devendran 

1. Store the summary, POS data in your json poet data structure and save to disk.

2. Interchange the verbs and adjectives of the two poets and 

3. save the data structure for the new poems:  poet1-poet2.json (replace poet1 adjectives and verbs with poet2's ) and poet2-poet1.json out the new poems and save them to disk poet1-poet2.txt and poet2-poet1.txt [these files will contain 10 poems each)].

* Deesha Desai

1. Combine poet1's and poet2's poem text and summarize it using the transformer

2. Run topic modeling on the combined text of each poet: what are the top 5 topics they are writing about?

3. Name the topics: use the new notebook : name_your_topic_background.ipynb (Links to an external site.) , to try and find the closest word that describes topic 0, topic 1, topic 2 : instead of topic 0, what is the closest word in that cluster that describes the topic?




In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 7.3 MB/s 
     |████████████████████████████████| 596 kB 48.1 MB/s 
     |████████████████████████████████| 3.3 MB 41.7 MB/s 
     |████████████████████████████████| 61 kB 517 kB/s 
     |████████████████████████████████| 895 kB 49.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Scraping poems for Alexander Pope & Weldon Kees

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp=en_core_web_sm.load()
nltk.download('punkt')
!pip install gensim==3.8.1
import gensim

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 24.2 MB 81.9 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import pandas as pd

In [ ]:
from __future__ import print_function
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import html
import pandas as pd
import json
import numpy as np


poetList = ["weldon-kees", "alexander-pope"]

complete_data = {}
df = pd.DataFrame(columns = ['poet','url','poem'])

def get_poem(url):
  hdr = {'User-Agent':'Mozilla/5.0'}
  req = Request(url,headers=hdr)
  page = urlopen(req)
  soup = BeautifulSoup(page.read(), 'html.parser')
  divs = soup.find_all('div', attrs={'style': 'text-indent: -1em; padding-left: 1em;'})
  text=''
  for div in divs:
      text+=div.text
  return text

data=[]
for poet in poetList:
    poet = poet.lower()
    poet = re.sub('[^a-z]+','-',poet)
    #fileout = poet + ".txt"
    #output = open(fileout,'w')

    url = "https://www.poetryfoundation.org/poets/"+poet+"#about"
    hdr = {'User-Agent':'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page.read(), 'html.parser')

    poems = soup.find_all('a',href=re.compile('.*/poems/[0-9]+/.*'))
    poems2 = soup.find_all('a',href=re.compile('.*/poem/.*'))

    poems.extend(poems2)
    poems = list(set(poems))
    for poem in poems:
        poemURL = poem.get('href')
        poem_text=get_poem(poemURL)
        if poem_text is not None:
            data.append([poet,poemURL,poem_text])
    

In [ ]:
df=pd.DataFrame(data,columns=['poet','url','poem'])
df['poem'].replace('', np.nan, inplace=True)
df=df.dropna()
df['poem'] = df['poem'].str.replace('\r','')
df['poem'] = df['poem'].str.strip()
df['poemSentences']=df['poem'].apply(lambda x: len(x.split('.')))

In [ ]:
df.head()

,poet,url,poem,poemSentences
0,weldon-kees,https://www.poetryfoundation.org/poetrymagazin...,"Change, move, dead clock, that this fresh dayM...",4
4,weldon-kees,https://www.poetryfoundation.org/poems/47579/r...,The dog stops barking after Robinson has gone....,16
5,weldon-kees,https://www.poetryfoundation.org/poetrymagazin...,Plurality is all. I walk among the restaurants...,11
12,weldon-kees,https://www.poetryfoundation.org/poems/53029/1926,"The porchlight coming on again, Early November...",12
14,weldon-kees,https://www.poetryfoundation.org/poems/47574/f...,Looking into my daughter’s eyes I read Bene...,8


In [ ]:
df=df[df['poemSentences']>6]
df.head()

,poet,url,poem,poemSentences
4,weldon-kees,https://www.poetryfoundation.org/poems/47579/r...,The dog stops barking after Robinson has gone....,16
5,weldon-kees,https://www.poetryfoundation.org/poetrymagazin...,Plurality is all. I walk among the restaurants...,11
12,weldon-kees,https://www.poetryfoundation.org/poems/53029/1926,"The porchlight coming on again, Early November...",12
14,weldon-kees,https://www.poetryfoundation.org/poems/47574/f...,Looking into my daughter’s eyes I read Bene...,8
15,weldon-kees,https://www.poetryfoundation.org/poems/47586/r...,"Curtains drawn back, the door ajar. All winter...",12


In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
weldon_kees=df[df.poet=='weldon-kees'].head(10)
alexander_pope=df[df.poet=='alexander-pope'].head(10)

In [ ]:
weldon_kees

,poet,url,poem,poemSentences
4,weldon-kees,https://www.poetryfoundation.org/poems/47579/robinson,"The dog stops barking after Robinson has gone. His act is over. The world is a gray world, Not without violence, and he kicks under the grand piano, The nightmare chase well under way. The mirror from Mexico, stuck to the wall, Reflects nothing at all. The glass is black. Robinson alone provides the image Robinsonian. Which is all of the room—walls, curtains, Shelves, bed, the tinted photograph of Robinson’s first wife, Rugs, vases, panatellas in a humidor. They would fill the room if Robinson came in. The pages in the books are blank, The books that Robinson has read. That is his favorite chair, Or where the chair would be if Robinson were here. All day the phone rings. It could be Robinson Calling. It never rings when he is here. Outside, white buildings yellow in the sun. Outside, the birds circle continuously Where trees are actual and take no holiday.",16
5,weldon-kees,https://www.poetryfoundation.org/poetrymagazine/poems/22320/statement-with-rhymes,"Plurality is all. I walk among the restaurants, the theatres, the grocery stores; I ride the cars and hear of Mrs. Bedford’s teeth and Albuquerque, strikes unsettled, someone’s simply marvelous date, news of the German Jews, the baseball scores, storetalk and whoretalk, talk of wars. I turn the pages of a thousand books to read the names of Buddha, Malthus, Walker Evans, Stendhal, André Gide, Ouspenski; note the terms: obscurantism, factorize, fagaceous, endocarp; descend the nervous stairs to hear the broken ends of songs that float through city air. In Osnabrück and Ogden, on the Passamaquoddy Bay, in Ahmednagar, Waco (Neb.), in Santa Fé, propelled by zeros, zinc, and zephyrs, always I’m pursued by thoughts of what I am, authority, remembrance, food, the letter on the mezzanine, the unemployed, dogs’ lonely faces, pianos and decay. Plurality is all. I sympathize, but cannot grieve too long for those who wear their dialectics on their sleeves. The pattern’s one I sometimes rather like; there’s really nothing wrong with it for some. But I should add: It doesn’t wear for long, before I push the elevator bell and quickly leave.",11
12,weldon-kees,https://www.poetryfoundation.org/poems/53029/1926,"The porchlight coming on again, Early November, the dead leaves Raked in piles, the wicker swing Creaking. Across the lots A phonograph is playing Ja-Da. An orange moon. I see the lives Of neighbors, mapped and marred Like all the wars ahead, and R. Insane, B. with his throat cut, Fifteen years from now, in Omaha. I did not know them then. My airedale scratches at the door. And I am back from seeing Milton Sills And Doris Kenyon. Twelve years old. The porchlight coming on again.",12
14,weldon-kees,https://www.poetryfoundation.org/poems/47574/for-my-daughter,"Looking into my daughter’s eyes I read Beneath the innocence of morning flesh Concealed, hintings of death she does not heed. Coldest of winds have blown this hair, and mesh Of seaweed snarled these miniatures of hands; The night’s slow poison, tolerant and bland, Has moved her blood. Parched years that I have seen That may be hers appear: foul, lingering Death in certain war, the slim legs green. Or, fed on hate, she relishes the sting Of others’ agony; perhaps the cruel Bride of a syphilitic or a fool. These speculations sour in the sun. I have no daughter. I desire none.",8
15,weldon-kees,https://www.poetryfoundation.org/poems/47586/robinson-at-home,"Curtains drawn back, the door ajar. All winter long, it seemed, a darkening Began. But now the moonlight and the odors of the street Conspire and combine toward one community. These are the rooms of Robinson. Bleached, wan, and colorless this light, as though All the blurred daybreaks of the spring Found an asylum here, perhaps for Robinson alone, Who sleeps. Were there more music sifted through the floors And moonlight of a different kind, He might awake to hear the news at ten, Which 

In [ ]:
alexander_pope

poet  ... poemSentences
47  alexander-pope  ...            41
48  alexander-pope  ...            81
50  alexander-pope  ...            36
51  alexander-pope  ...            87
52  alexander-pope  ...            36
55  alexander-pope  ...            84
56  alexander-pope  ...            39
57  alexander-pope  ...            10
58  alexander-pope  ...            72
59  alexander-pope  ...            45

[10 rows x 4 columns]

## Saving JSON data of poems to json file

In [ ]:
weldon_kees.to_json(r'weldon_kees.json',orient='records')
alexander_pope.to_json(r'alexander_pope.json',orient='records')


#Run POS and extract verbs, nouns and adjectives

In [ ]:
import nltk
nltk.download('punkt') 
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
from nltk.tokenize import word_tokenize
def get_pos(poet):
  global_vrb= []
  global_nns= []
  global_adj= []
  poems=poet['poem']
  for val in  (poems):
    wrd_list = word_tokenize(val)
    pos_val = nltk.pos_tag(wrd_list)
    vrb = set([word for (word, pos) in pos_val if (pos.startswith('VB'))])
    nns = set([word for (word, pos) in pos_val if (pos.startswith('NN'))])
    adj = set([word for (word, pos) in pos_val if (pos.startswith('JJ'))])

    global_vrb.append(list(vrb))
    global_nns.append(list(nns))
    global_adj.append(list(adj))
    
  return global_vrb, global_nns, global_adj

In [ ]:
weldon_kees_verbs,weldon_kees_nns,weldon_kees_adj=get_pos(weldon_kees)

print(weldon_kees_verbs)
print(weldon_kees_nns)
print(weldon_kees_adj)

[['has', 'kicks', 'Reflects', 'are', 'stuck', 'were', 'rings', 'stops', 'read', 'fill', 'be', 's', 'gone', 'take', 'came', 'is', 'provides', 'barking'], ['note', 'faces', 'read', 'turn', 's', 'sympathize', 'am', 'factorize', 'doesn', 'unsettled', 'hear', 'wear', 'walk', 'propelled', 'pursued', 'ride', 'is', '’', 'push', 'descend', 'add', 'grieve', 'float', 'leave'], ['Raked', 'know', 'coming', 'playing', 'marred', 'seeing', 'see', 'am', 'is', 'did', 'mapped', 'swing'], ['have', 'moved', 'appear', 'relishes', 'blown', 'seen', 'heed', 'sour', 'read', 'desire', 'Looking', 'be', 's', 'lingering', 'fed', 'does', 'Parched', 'snarled'], ['bearded', 'awake', 'seemed', 'Found', 'mumbles', 'known', 'hear', 'shocking', 'die', 'wear', 'turns', 'has', 'are', '”', 'wakes', 'symbolize—', 'blow', 'be', 'sleeps', 'sifted', 'drawn', '“', 'is', 'drones', 'combine'], ['Moving', 'wake', 'become', 'hang', 'unmask', 'murmurs', 'remembering', 'fallen', 'descends', 'held', 'eat', 'barred', 'lifted', 'talked', 

In [ ]:
alexander_pope_verbs,alexander_pope_nns,alexander_pope_adj=get_pos(alexander_pope)

print(alexander_pope_verbs)
print(alexander_pope_nns)
print(alexander_pope_adj)

[['learn', 'please', 'aside', 'alone', 'mend', 'zealous', 'rears', 'born', 'give', 'See', 'reddens', 'did', 'draws', "list'ning", 'stood', 'degrees.Leave', 'speak', 'left', 'steal', 'bear', 'judge', 'taught', 'strengthens', 'serve', 'name', 'works', 'wants', 'failings', 'boast', 'rang', 'rising', 'begun', 'dull', 'knew', 'reign', 'offend', 'seeming', 'defied', 'see', 'make', 'Led', 'renew', 'saw', 'forth', 'let', 'ordain', 'bestow', 'fulsome', 'turn', 'looks', 'truths', 'makes', "thund'ring", 'know', 'rise', 'follow', 'doubt', 'proposed', "ev'ry", 'rattling', 'sung', 'wit', 'appears.All', 'edifies', 'was', 'love', 'be', 'leap', "'t", 'Be', 'Starts', 'Bursts', 'grown', 'durst', 'asleep.False', 'do', 'bless', 'began', 'writ', 'stumbling', 'raise', 'scribbling', 'write', 'charms', 'preside', 'were', 'despis', 'threatening', 'place', 'bays', 'findThe', 'succeeding', 'are', 'prun', 'allow', 'prove', 'sing', 'knew.The', 'steer', 'live', 'gives', 'commend', 'praise', 'run', 'form', 'tread.Dis

**Saving the pos data for Alexander Pope inn json format**

In [ ]:
alexander_pos=weldon_kees
alexander_pos['verbs']=alexander_pope_verbs
alexander_pos['nns']=alexander_pope_nns
alexander_pos['adj']=alexander_pope_adj
alexander_pos.head()
alexander_pos.to_json(r'alexander_pos.json',orient='records')
files.download('alexander_pos.json') 
alexander_pos.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,poet,url,poem,poemSentences,verbs,nns,adj
4,weldon-kees,https://www.poetryfoundation.org/poems/47579/robinson,"The dog stops barking after Robinson has gone. His act is over. The world is a gray world, Not without violence, and he kicks under the grand piano, The nightmare chase well under way. The mirror from Mexico, stuck to the wall, Reflects nothing at all. The glass is black. Robinson alone provides the image Robinsonian. Which is all of the room—walls, curtains, Shelves, bed, the tinted photograph of Robinson’s first wife, Rugs, vases, panatellas in a humidor. They would fill the room if Robinson came in. The pages in the books are blank, The books that Robinson has read. That is his favorite chair, Or where the chair would be if Robinson were here. All day the phone rings. It could be Robinson Calling. It never rings when he is here. Outside, white buildings yellow in the sun. Outside, the birds circle continuously Where trees are actual and take no holiday.",16,"[learn, please, aside, alone, mend, zealous, rears, born, give, See, reddens, did, draws, list'ning, stood, degrees.Leave, speak, left, steal, bear, judge, taught, strengthens, serve, name, works, wants, failings, boast, rang, rising, begun, dull, knew, reign, offend, seeming, defied, see, make, Led, renew, saw, forth, let, ordain, bestow, fulsome, turn, looks, truths, makes, thund'ring, know, rise, follow, doubt, proposed, ev'ry, rattling, sung, wit, appears.All, edifies, was, love, be, leap, 't, Be, Starts, Bursts, grown, durst, asleep.False, do, bless, began, writ, stumbling, raise, scribbling, write, charms, preside, were, despis, threatening, place, bays, findThe, succeeding, are, prun, allow, prove, sing, knew.The, steer, live, ...]","[tide, kind, Content, proud, practice, taste, mischief, Who, Greece, name, old.Yet, converse, Dryden, resistless, knew, desert, head, Dionysius, course, numbers, favour, grace, rules, shame, faults, Will, Learn, bards, poet, torrent, trance, notions, critic, lumber, Licence, Blunt, avarice, Might, Tales.With, Romans, Cremona, revive, critics, churchyard, speaks, mild, sways.But, sense, morals, Prescrib, tax, Shakes, arms, Vandals, crowds, Muse, shade, fle'me, beauties, Learning, genius, tops, Britons, fly, laws, tries, liberty, stage, guide, home, counsel, man, honour, dust, Rome, order, Thence, fewOf, scholar, example, right, suffers, way.He, sounds, books, work, breeding, merit, learn, please, sweeter, droppings, dedicators, Goths, thoughts, Petronius, rush, judge, fame, fury, ...]","[severe, superior, ancient, zealous, learned, banished, late, sure, shameless, short, proud, true, grateful, critic-learning, nice, Mæonian, fierce, lash, finish, own.Such, rev'rend, clearest, worst, uncensur, dull, impious, chief, pleas, northern, human, afraid, silent, grave, uncivilis, graceful, presum, unjust.Fear, bookful, well-bred, better, sincerest, ev'ry, conveyThe, civil, believ, warmth, wrong, wild, most, ready, sister-arts, humble, Such, constru, least, Gen'rous, great, less, unconfin, happy, justest, full, vain, Tremendous, dangerous, fundamental, disapprov, good, own, mad, fire.An, unsuccessful, drowsy, little, new, positive, safe, due, judg, mean, noble, last, early, best, barb'rous, useful, copious, truest, gen'rous, honourable, second, prepossess, free, easiest, contrary, dead, sacred, ivy, same, candour, ...]"
5,weldon-kees,https://www.poetryfoundation.org/poetrymagazine/poems/22320/statement-with-rhymes,"Plurality is all. I walk among the restaurants, the theatres, the grocery stores; I ride the cars and hear of Mrs. Bedford’s teeth and Albuquerque, strikes unsettled, someone’s simply marvelous date, news of the German Jews, the baseball scores, storetalk and whoretalk, talk of wars. I turn the pages of a thousand books to read the names of Buddha, Malthus, Walker Evans, Stendhal, André Gide, Ouspenski; note the terms: obscurantism, factorize, fagaceous, endocarp; descend the nervous stairs to hear the broken ends of songs 

**Saving the pos data for Weldon kees inn json format**

In [ ]:
weldon_kees_pos=weldon_kees
weldon_kees_pos['verbs']=weldon_kees_verbs
weldon_kees_pos['nns']=weldon_kees_nns
weldon_kees_pos['adj']=weldon_kees_adj
weldon_kees_pos.head()
weldon_kees_pos.to_json(r'weldon_kees_pos.json',orient='records')
files.download('weldon_kees_pos.json') 
weldon_kees_pos.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,poet,url,poem,poemSentences,verbs,nns,adj
4,weldon-kees,https://www.poetryfoundation.org/poems/47579/robinson,"The dog stops barking after Robinson has gone. His act is over. The world is a gray world, Not without violence, and he kicks under the grand piano, The nightmare chase well under way. The mirror from Mexico, stuck to the wall, Reflects nothing at all. The glass is black. Robinson alone provides the image Robinsonian. Which is all of the room—walls, curtains, Shelves, bed, the tinted photograph of Robinson’s first wife, Rugs, vases, panatellas in a humidor. They would fill the room if Robinson came in. The pages in the books are blank, The books that Robinson has read. That is his favorite chair, Or where the chair would be if Robinson were here. All day the phone rings. It could be Robinson Calling. It never rings when he is here. Outside, white buildings yellow in the sun. Outside, the birds circle continuously Where trees are actual and take no holiday.",16,"[has, kicks, Reflects, are, stuck, were, rings, stops, read, fill, be, s, gone, take, came, is, provides, barking]","[trees, piano, wife, rings, mirror, way, pages, sun, nightmare, Outside, Mexico, buildings, holiday, glass, room, circle, dog, bed, wall, image, violence, yellow, Shelves, chase, photograph, Which, phone, Robinsonian, Robinson, day, curtains, panatellas, chair, vases, ’, Rugs, humidor, world, birds, books, act, nothing, Calling, room—walls]","[blank, gray, tinted, favorite, grand, first, black, white, actual]"
5,weldon-kees,https://www.poetryfoundation.org/poetrymagazine/poems/22320/statement-with-rhymes,"Plurality is all. I walk among the restaurants, the theatres, the grocery stores; I ride the cars and hear of Mrs. Bedford’s teeth and Albuquerque, strikes unsettled, someone’s simply marvelous date, news of the German Jews, the baseball scores, storetalk and whoretalk, talk of wars. I turn the pages of a thousand books to read the names of Buddha, Malthus, Walker Evans, Stendhal, André Gide, Ouspenski; note the terms: obscurantism, factorize, fagaceous, endocarp; descend the nervous stairs to hear the broken ends of songs that float through city air. In Osnabrück and Ogden, on the Passamaquoddy Bay, in Ahmednagar, Waco (Neb.), in Santa Fé, propelled by zeros, zinc, and zephyrs, always I’m pursued by thoughts of what I am, authority, remembrance, food, the letter on the mezzanine, the unemployed, dogs’ lonely faces, pianos and decay. Plurality is all. I sympathize, but cannot grieve too long for those who wear their dialectics on their sleeves. The pattern’s one I sometimes rather like; there’s really nothing wrong with it for some. But I should add: It doesn’t wear for long, before I push the elevator bell and quickly leave.",11,"[note, faces, read, turn, s, sympathize, am, factorize, doesn, unsettled, hear, wear, walk, propelled, pursued, ride, is, ’, push, descend, add, grieve, float, leave]","[grocery, Neb, Waco, zephyrs, letter, someone, Buddha, decay, stores, pages, s, scores, city, Evans, André, thoughts, teeth, Ogden, songs, restaurants, sleeves, mezzanine, dialectics, elevator, strikes, Fé, cars, hear, news, wars, Santa, dogs, talk, Ahmednagar, Stendhal, pattern, wear, whoretalk, obscurantism, Bay, Passamaquoddy, ends, remembrance, date, Malthus, Ouspenski, authority, theatres, Mrs., zeros, Walker, thousand, Gide, ’, Plurality, terms, Bedford, Jews, air, Albuquerque, t, books, pianos, storetalk, nothing, Osnabrück, bell, baseball, food, names, stairs, zinc]","[nervous, endocarp, wrong, marvelous, m, German, broken, unemployed, fagaceous, ’]"
12,weldon-kees,https://www.poetryfoundation.org/poems/53029/1926,"The porchlight coming on again, Early November, the dead leaves Raked in piles, the wicker swing Creaking. Across the lots A phonograph is playing Ja-Da. An orange moon. I see the lives Of neighbors, mapped and marred Like all the wars ahead, and R. Insane, B. with his throat cut, Fifteen years from now, in Omaha. I did not know them

# Summarize each poem 

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
import os


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [ ]:
def get_transformer_summary(text):

  summary_text_1 = summarizer(text, max_length=512, min_length=5, do_sample=False)[0]['summary_text']
  return summary_text_1

In [ ]:
weldon_kees['summary']=weldon_kees['poem'].apply(lambda x: get_transformer_summary(x))

Your max_length is set to 512, but you input_length is only 138. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 512, but you input_length is only 165. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 512, but you input_length is only 330. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 512, but you input_length is only 188. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 512, but you input_length is only 329. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 512, but you input_length is only 229. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 512, but you input_length is only 145. You might c

In [ ]:
weldon_kees

In [ ]:
from google.colab import files

In [ ]:
alexander_pope['summary']=alexander_pope['poem'].apply(lambda x: get_transformer_summary(x))

Token indices sequence length is longer than the specified maximum sequence length for this model (5146 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
alexander_pope

poet  ...                                                                                                                                                                                                                                                                                                 summary
47  alexander-pope  ...                                                                                     elisabeth mccarthy: if thou wilts, thy love will be redeemed . she says if she drew to heaven, she would be a saint, but she would not be . cctv: tuesday, october 2, at 9 p.m. et, at 11 a.m, at 7 p'm., at 10 a'm
49  alexander-pope  ...                           david rothkopf: whoever rejects mankind, is by some sylph embrac'd . he says nymphs swell their prospects, exalt their pride, when offers are disdain'd and love denied . but he writes: if ev'n belinda slept too long, he woke up and kissed his mistress with his tongue .
50  alexander-pope  ...                                            st. john's day is a day of remembrance and reflection . he asks why man is so blind, that he can't see beyond his own sphere . but if he errs, he'll be able to see beyond what he sees and sees . and he says he will not be a slave to a god, but a king .
51  alexander-pope  ...                                                                                                                                       'tis strange, the prodigal should wasteHis wealth to purchase what he ne'er can taste?' he says . 'it's not for himself, but for his wife, alas! or finer whore .
52  alexander-pope  ...                cnn's ireport boot camp tackles a sex scandal involving a pow'r . frida ghitis: if e'er thy gnome could spoil a grace, he'd be the first mistaken maid . she says if she had snatched her hair, she'd have a chance to win back her honour . ghittingis: she'll be a great hero, but she
53  alexander-pope  ...                 david frum says heav'n decreed with prude: "beauties in vain their pretty eyes may roll" he says a hero's wits are kept in pond'rous vases and beaux' in snuff boxes and tweezercases . frum: a nymph's death is a symbol of a man's love for a woman, and a god's desire to save her .
54  alexander-pope  ...                                                                           cnn's john sutter writes a letter to a friend of his own . he says he was not born to be a poet, but to be one of a kind . "to live and die is all that I have to do," he writes . but he is not a fool; he's just a savage .
55  alexander-pope  ...                            'tis not enough, your counsel still be true;Blunt truths more mischief than nice falsehoods do;Men must be taught as if you taught them not' 'Tis best sometimes your censure to restrain, and charitably let the dull be vain:Your silence there is better than your spite'
56  alexander-pope  ...                                                                                                                                 thou, false guardian of a charge too good, mean deserter of thy brother's blood . if eternal justice rules the ball,thus shall your wives and thus your children fall .
58  alexander-pope  ...  cnn's timothy stanley: kings, queens, and nymphs are all tyrannized by fate . he says they are tainted by fate, but they are not rewarded . she says a king, a queen, or a prince, are rewarded for their bravery and bravery . writer: if a man is rewarded, he will be rewarded with a royal crown .

[10 rows x 5 columns]

# Store the summary, POS data in your json poet data structure and save to disk

In [ ]:
alexander_pope.to_json(r'alexander_pope_summary.json',orient='records')

In [ ]:
files.download('alexander_pope_summary.json') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
weldon_kees.to_json(r'weldon_kees_summary.json',orient='records')

In [ ]:
files.download('weldon_kees_summary.json') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Interchange the verbs and adjectives of the two poets

In [ ]:
!pip install gensim==3.8.1
import gensim

In [ ]:
def closestWord(words):
    closest = max(words,key=lambda item:item[1])
    return closest
def Findreplacement(word, model):
  if word in list(model.wv.vocab.keys()):
    mdl = model.wv.most_similar(positive=word)
    matched = closestWord(mdl)
    if matched[1] > 0.1:
      return matched[0]
    else:
      return word
  else:
    return word
def swapWords(words, text, model):
    for w in words:
      word_repl = Findreplacement(w, model)
      if w!=word_repl:
        print("word "+ w +' is replaced by ' + word_repl)
      text = text.replace(' '+ w +' ', ' ' + word_repl +' ')
    return text

In [ ]:
def swap_adjVrbs(poet,adj,verbs,model_vrb, model_adj):
    new_poem=[]
    poems=poet['poem']
    for i,text in enumerate (poems):
      new_text = swapWords(verbs[i], text, model_vrb)
      new_text1 = swapWords(adj[i], new_text, model_adj)
      new_poem.append([new_text1])
    return new_poem

In [ ]:
model_alex_adj = gensim.models.Word2Vec(alexander_pope_adj, min_count=1)
model_alex_adj.train(alexander_pope_adj,total_examples=len(alexander_pope_adj),epochs=10)
model_alex_vrb = gensim.models.Word2Vec(alexander_pope_verbs, min_count=1)
model_alex_vrb.train(alexander_pope_verbs,total_examples=len(alexander_pope_verbs),epochs=10)

model_stan_adj = gensim.models.Word2Vec(weldon_kees_adj, min_count=1)
model_stan_adj.train(weldon_kees_adj,total_examples=len(weldon_kees_adj),epochs=10)
model_stan_vrb = gensim.models.Word2Vec(weldon_kees_verbs, min_count=1)
model_stan_vrb.train(weldon_kees_verbs,total_examples=len(weldon_kees_verbs),epochs=10)

(896, 1760)

In [ ]:
weldon_inter_poem1=swap_adjVrbs(weldon_kees,weldon_kees_adj,weldon_kees_verbs, model_alex_vrb, model_alex_adj)

word know is replaced by proposed
word did is replaced by th
word see is replaced by wasteHis
word am is replaced by gazing
word is is replaced by adds
word old is replaced by unveil
word dead is replaced by appearsAlready
word seen is replaced by freed
word appear is replaced by guide
word be is replaced by speaks
word have is replaced by discompos
word read is replaced by Give
word green is replaced by let
word certain is replaced by routed
word add is replaced by gushing
word descend is replaced by repent
word turn is replaced by bring
word leave is replaced by Oh
word is is replaced by adds
word grieve is replaced by sexes
word am is replaced by gazing
word walk is replaced by suffer
word read is replaced by Give
word hear is replaced by leave
word wrong is replaced by transient
word broken is replaced by eternal
word left is replaced by lock
word thought is replaced by view
word were is replaced by swell
word was is replaced by insult
word died is replaced by 'd
word is is replace

In [ ]:
alex_inter_poem2=swap_adjVrbs(alexander_pope,alexander_pope_adj,alexander_pope_verbs, model_stan_vrb,model_stan_adj)

word turn is replaced by fed
word grieve is replaced by stops
word has is replaced by wakes
word be is replaced by came
word was is replaced by bathing
word died is replaced by Should
word winds is replaced by know
word saw is replaced by kicks
word dying is replaced by arranges
word thought is replaced by Coming
word have is replaced by doesn
word are is replaced by walking
word had is replaced by Huddled
word came is replaced by Walked
word take is replaced by add
word see is replaced by unsettled
word is is replaced by Raked
word were is replaced by see
word moving is replaced by wipe
word heard is replaced by Huddled
word blow is replaced by ’
word left is replaced by heed
word read is replaced by descend
word hear is replaced by leave
word blank is replaced by marvelous
word bright is replaced by bad
word first is replaced by favorite
word silent is replaced by airedale
word more is replaced by present
word human is replaced by favorite
word green is replaced by slim
word white is

## save the data structure for the new poems:  poet1-poet2.json (replace poet1 adjectives and verbs with poet2's ) and poet2-poet1.json 

**Storing the swapped poems of alexander_pope in json format**

In [ ]:
weldon_kees_new=weldon_kees
weldon_kees_new['swapped_poem']=weldon_inter_poem1
weldon_kees_new.head()
weldon_kees_new.to_json(r'weldon_alexander.json',orient='records')
# files.download('weldon_alexander.json') 
weldon_kees_new.head()

,poet,url,poem,poemSentences,summary,swapped_poem
1,weldon-kees,https://www.poetryfoundation.org/poems/53029/1926,"The porchlight coming on again, Early November, the dead leaves Raked in piles, the wicker swing Creaking. Across the lots A phonograph is playing Ja-Da. An orange moon. I see the lives Of neighbors, mapped and marred Like all the wars ahead, and R. Insane, B. with his throat cut, Fifteen years from now, in Omaha. I did not know them then. My airedale scratches at the door. And I am back from seeing Milton Sills And Doris Kenyon. Twelve years old. The porchlight coming on again.",12,"the porchlight coming on again, early November, the dead leaves Raked in piles . john sutter: i see the lives Of neighbors, mapped and marred Like all the wars ahead .","[The porchlight coming on again, Early November, the appearsAlready leaves Raked in piles, the wicker swing Creaking. Across the lots A phonograph adds playing Ja-Da. An orange moon. I wasteHis the lives Of neighbors, mapped and marred Like all the wars ahead, and R. Insane, B. with his throat cut, Fifteen years from now, in Omaha. I th not proposed them then. My airedale scratches at the door. And I gazing back from seeing Milton Sills And Doris Kenyon. Twelve years old. The porchlight coming on again.]"
4,weldon-kees,https://www.poetryfoundation.org/poems/47574/for-my-daughter,"Looking into my daughter’s eyes I read Beneath the innocence of morning flesh Concealed, hintings of death she does not heed. Coldest of winds have blown this hair, and mesh Of seaweed snarled these miniatures of hands; The night’s slow poison, tolerant and bland, Has moved her blood. Parched years that I have seen That may be hers appear: foul, lingering Death in certain war, the slim legs green. Or, fed on hate, she relishes the sting Of others’ agony; perhaps the cruel Bride of a syphilitic or a fool. These speculations sour in the sun. I have no daughter. I desire none.",8,i have no daughter. I desire none.,"[Looking into my daughter’s eyes I read Beneath the innocence of morning flesh Concealed, hintings of death she does not heed. Coldest of winds discompos blown this hair, and mesh Of seaweed snarled these miniatures of hands; The night’s slow poison, tolerant and bland, Has moved her blood. Parched years that I discompos seen That may speaks hers appear: foul, lingering Death in routed war, the slim legs green. Or, fed on hate, she relishes the sting Of others’ agony; perhaps the cruel Bride of a syphilitic or a fool. These speculations sour in the sun. I discompos no daughter. I desire none.]"
5,weldon-kees,https://www.poetryfoundation.org/poetrymagazine/poems/22320/statement-with-rhymes,"Plurality is all. I walk among the restaurants, the theatres, the grocery stores; I ride the cars and hear of Mrs. Bedford’s teeth and Albuquerque, strikes unsettled, someone’s simply marvelous date, news of the German Jews, the baseball scores, storetalk and whoretalk, talk of wars. I turn the pages of a thousand books to read the names of Buddha, Malthus, Walker Evans, Stendhal, André Gide, Ouspenski; note the terms: obscurantism, factorize, fagaceous, endocarp; descend the nervous stairs to hear the broken ends of songs that float through city air. In Osnabrück and Ogden, on the Passamaquoddy Bay, in Ahmednagar, Waco (Neb.), in Santa Fé, propelled by zeros, zinc, and zephyrs, always I’m pursued by thoughts of what I am, authority, remembrance, food, the letter on the mezzanine, the unemployed, dogs’ lonely faces, pianos and decay. Plurality is all. I sympathize, but cannot grieve too long for those who wear their dialectics on their sleeves. The pattern’s one I sometimes rather like; there’s really nothing wrong with it for some. But I should add: It doesn’t wear for long, before I push the elevator bell and quickly leave.",11,"in osnabrück and ogden, on the passamaquoddy bay, in Ahmednagar, waco, in Santa Fé, he’s pursued by thoughts of what he is . he sympathizes, but cannot grieve too long for

**Storing the swapped poems of alexander_pope in json format**

In [ ]:
alexander_pope_new=alexander_pope
alexander_pope_new['swapped_poem']=alex_inter_poem2
alexander_pope_new.head()
alexander_pope_new.to_json(r'alexander_weldon.json',orient='records')
files.download('alexander_weldon.json') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## print out the new poems and save them to disk poet1-poet2.txt and poet2-poet1.txt [these files will contain 10 poems each)].

**Replaced Weldon's adjectives and verbs with Alex's**


In [ ]:
for poem in (weldon_inter_poem1):
  print(poem)

['The porchlight coming on again, Early November, the appearsAlready leaves Raked in piles, the wicker swing Creaking. Across the lots A phonograph adds playing Ja-Da. An orange moon. I wasteHis the lives Of neighbors, mapped and marred Like all the wars ahead, and R. Insane, B. with his throat cut, Fifteen years from now, in Omaha. I th not proposed them then. My airedale scratches at the door. And I gazing back from seeing Milton Sills And Doris Kenyon. Twelve years old. The porchlight coming on again.']
['Looking into my daughter’s eyes I read\xa0\xa0\xa0 Beneath the innocence of morning flesh\xa0\xa0\xa0 Concealed, hintings of death she does not heed. Coldest of winds discompos blown this hair, and mesh Of seaweed snarled these miniatures of hands; The night’s slow poison, tolerant and bland, Has moved her blood. Parched years that I discompos seen\xa0\xa0\xa0 That may speaks hers appear: foul, lingering\xa0\xa0\xa0 Death in routed war, the slim legs green.\xa0\xa0\xa0 Or, fed on h

**Storing weldon_alex.txt file**

In [ ]:
weldon = np.array(weldon_inter_poem1)
file = open("weldon_alex.txt", "w+")
content = str(weldon)
file.write(content)
file.close()
files.download('weldon_alex.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Storing alex_weldon.txt file**

In [ ]:
alex = np.array(alex_inter_poem2)
file = open("alex_weldon.txt", "w+")
content = str(alex)
file.write(content)
file.close()
files.download('alex_weldon.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Replaced Alex's adjectives and verbs with Weldons's**


In [ ]:
for poem in (alex_inter_poem2):
  print(poem)

['In these deep solitudes and awful cells,Where heav\'nly-pensive contemplation dwells,And ever-musing melancholy reigns;What means this tumult in a vestal\'s veins?Why rove my thoughts beyond this last retreat?Why feels my heart its long-forgotten heat?Yet, yet I love!—From Abelard it came,And Eloisa yet must kiss the name.      Dear fatal name! rest ever unreveal\'d,Nor pass these lips in holy silence seal\'d.Hide it, my heart, within that close disguise,Where mix\'d with God\'s, his lov\'d idea lies:O write it not, my hand—the name appearsAlready written—wash it out, my tears!In vain lost Eloisa weeps and prays,Her heart still dictates, and her hand obeys.      Relentless walls! whose darksome round containsRepentant sighs, and voluntary pains:Ye rugged rocks! which holy knees doesn worn;Ye grots and caverns shagg\'d with horrid thorn!Shrines! where their vigils pale-ey\'d virgins keep,And pitying saints, whose statues learn to weep!Though cold like you, unmov\'d, and airedale grown

# Combine poet1's and poet2's poem text and summarize it using the transformer

In [ ]:
combined_text_weldon_kees = ""
for poem in weldon_kees['poem']:
  combined_text_weldon_kees += str(poem)
print(combined_text_weldon_kees)

Somewhere in Chelsea, early summer; And, walking in the twilight toward the docks,    I thought I made out Robinson ahead of me. From an uncurtained second-story room, a radio    Was playing There’s a Small Hotel; a kite Twisted above dark rooftops and slow drifting birds.    We were alone there, he and I, Inhabiting the empty street. Under a sign for Natural Bloom Cigars, While lights clicked softly in the dusk from red to green,    He stopped and gazed into a window Where a plaster Venus, modeling a truss, Looked out at Eastbound traffic. (But Robinson, I knew, was out of town: he summers at a place in Maine,    Sometimes on Fire Island, sometimes the Cape, Leaves town in June and comes back after Labor Day.) And yet, I almost called out, “Robinson!” There was no chance. Just as I passed,    Turning my head to search his face,    His own head turned with mine And fixed me with dilated, terrifying eyes    That stopped my blood. His voice Came at me like an echo in the dark. “I thought

In [ ]:
combined_text_alexeder_pope = ""
for poem in alexander_pope['poem']:
  combined_text_alexeder_pope += str(poem)
print(combined_text_alexeder_pope)

Close by those meads, for ever crown'd with flow'rs, Where Thames with pride surveys his rising tow'rs, There stands a structure of majestic frame, Which from the neighb'ring Hampton takes its name. Here Britain's statesmen oft the fall foredoom Of foreign tyrants and of nymphs at home; Here thou, great Anna! whom three realms obey, Dost sometimes counsel take—and sometimes tea.        Hither the heroes and the nymphs resort, To taste awhile the pleasures of a court; In various talk th' instructive hours they pass'd, Who gave the ball, or paid the visit last; One speaks the glory of the British queen, And one describes a charming Indian screen; A third interprets motions, looks, and eyes; At ev'ry word a reputation dies. Snuff, or the fan, supply each pause of chat, With singing, laughing, ogling, and all that.        Meanwhile, declining from the noon of day, The sun obliquely shoots his burning ray; The hungry judges soon the sentence sign, And wretches hang that jury-men may dine; T

In [ ]:
!pip install transformers
from transformers import pipeline
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


In [ ]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
%%time
summary_text_1 = summarizer(combined_text_weldon_kees, max_length=2194, min_length=5, do_sample=False)[0]['summary_text']
print(summary_text_1)

Token indices sequence length is longer than the specified maximum sequence length for this model (3682 > 512). Running this sequence through the model will result in indexing errors


cnn's john sutter recalls a time when he thought he saw a whirlpool opening, kicked it all night . he wondered if it was a woman, or a man, or someone else, or something else . "it is late tonight as it will ever be," he writes . but he doesn't know if he's alive or dead; he just wants to be alive .
CPU times: user 3min 30s, sys: 7.84 s, total: 3min 37s
Wall time: 2min 13s


In [ ]:
%%time
summary_text_2 = summarizer(combined_text_alexeder_pope, max_length=2769, min_length=5, do_sample=False)[0]['summary_text']
print(summary_text_2)

# Run topic modeling on the combined text of each poet: Using LatentDirichletAllocation

## Poet 1: Weldon Kees

In [ ]:
weldon_kees_df=pd.read_json('/content/weldon_kees.json')

In [ ]:
weldon_kees_df

In [ ]:
weldon_kees_text=weldon_kees_df.poem.str.cat(sep=', ')

In [ ]:
weldon_kees_text

In [ ]:
import re
weldon_kees_text=re.sub('[^A-Za-z0-9 ]+', '', weldon_kees_text)

In [ ]:
weldon_kees_text


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
import numpy as np
import seaborn as sns

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
# Initialise the count vectorizer with the English stop words
count_vectorizer = CountVectorizer(stop_words=stopwords)
# Fit and transform the processed titles
count_data = count_vectorizer.fit_transform(weldon_kees_df.poem)
l = []
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        l.append(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    
# Tweak the two parameters below
number_topics = 5
number_words = 15
# Create and fit the LDA model
lda = LDA(n_components=number_topics, n_jobs=-1)
lda.fit(count_data)

weldon_kees_topics=l
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)

In [ ]:
def plot_10_most_common_words(count_data, count_vectorizer):
    import matplotlib.pyplot as plt
    words = count_vectorizer.get_feature_names()
    total_counts = np.zeros(len(words))
    for t in count_data:
        total_counts+=t.toarray()[0]
    
    count_dict = (zip(words, total_counts))
    count_dict = sorted(count_dict, key=lambda x:x[1], reverse=True)[0:10]
    words = [w[0] for w in count_dict]
    counts = [w[1] for w in count_dict]
    x_pos = np.arange(len(words)) 
    
    plt.figure(2, figsize=(15, 15/1.6180))
    plt.subplot(title='10 most common words')
    sns.set_context("notebook", font_scale=1.25, rc={"lines.linewidth": 2.5})
    sns.barplot(x_pos, counts, palette='husl')
    plt.xticks(x_pos, words, rotation=90) 
    plt.xlabel('words')
    plt.ylabel('counts')
    plt.show()
# Visualise the 10 most common words
plot_10_most_common_words(count_data, count_vectorizer)

## Poet 2: Alexander Pope

In [ ]:
alexander_pope_df=pd.read_json('/content/alexander_pope.json')

In [ ]:
alexander_pope_text=alexander_pope_df.poem.str.cat(sep=', ')

In [ ]:
import re
alexander_pope_text=re.sub('[^A-Za-z0-9 ]+', '', alexander_pope_text)

In [ ]:
alexander_pope_text

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
# Initialise the count vectorizer with the English stop words
count_vectorizer = CountVectorizer(stop_words=stopwords)
# Fit and transform the processed titles
count_data = count_vectorizer.fit_transform(alexander_pope_df.poem)
l = []

def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        l.append(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
# Tweak the two parameters below
number_topics = 5
number_words = 15
# Create and fit the LDA model
lda = LDA(n_components=number_topics, n_jobs=-1)
lda.fit(count_data)
alexander_pope_topics=l
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)

In [ ]:
plot_10_most_common_words(count_data, count_vectorizer)

# Name the topics: 

In [ ]:
weldon_kees_text

In [ ]:
import gensim
import nltk
nltk.download('brown')

In [ ]:
from nltk.corpus import brown
model = gensim.models.Word2Vec(brown.sents())

In [ ]:
model.save('brown.embedding')
new_model = gensim.models.Word2Vec.load('brown.embedding')

## Poet 1

In [ ]:
new_model.train(weldon_kees_text,total_examples=len(weldon_kees_text),epochs=10)

In [ ]:
weldon_kees_topics

In [ ]:
topic_x=[]
for topic in weldon_kees_topics:
  w = filter(lambda x: x in new_model.wv.vocab,topic.split(' '))
  x=new_model.wv.most_similar (positive=w)
  print(x)
  topic_x.append([topic,x[0][0]])
weldon_kees_topics_df=pd.DataFrame(topic_x)


In [ ]:
weldon_kees_topics_df.columns=['LDA Topics', 'Topic Name']

In [ ]:
weldon_kees_topics_df

## Poet 2:   

In [ ]:
new_model1 = gensim.models.Word2Vec.load('brown.embedding')

In [ ]:
new_model1.train(alexander_pope_text,total_examples=len(alexander_pope_text),epochs=10)

In [ ]:
alexander_pope_topics

In [ ]:
topic_x=[]
for topic in alexander_pope_topics:
  w = filter(lambda x: x in new_model1.wv.vocab,topic.split(' '))
  x=new_model1.wv.most_similar (positive=w)
  print(x)
  topic_x.append([topic,x[0][0]])
alexander_pope_topics_df=pd.DataFrame(topic_x)

In [ ]:
alexander_pope_topics_df.columns=['LDA Topics', 'Topic Name']

In [ ]:
alexander_pope_topics_df

# Run Topic Modelling using ldamodel from genism

In [ ]:
!pip install pyLDAvis
!pip install gensim
!pip install spacy==2.2.0
!pip install sentence_transformers

     |████████████████████████████████| 10.2 MB 5.1 MB/s 
     |████████████████████████████████| 2.1 MB 44.0 MB/s 
  Attempting uninstall: plac
    Found existing installation: plac 1.1.3
    Uninstalling plac-1.1.3:
      Successfully uninstalled plac-1.1.3
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 2.2.5 requires spacy>=2.2.2, but you have spacy 2.2.0 which is incompatible.
     |████████████████████████████████| 78 kB 3.2 MB/s 
     |████████████████████████████████| 3.1 MB 11.2 MB/s 
     |████████████████████████████████| 3.3 MB 21.6 MB/s 
     

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# Run in python console
import nltk; nltk.download('stopwords')

# Run in terminal or command prompt
!python3 -m spacy download en

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
from nltk.parse import load_parser
import spacy 
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
     |████████████████████████████████| 12.0 MB 4.4 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.0-py3-none-any.whl size=12019124 sha256=a0affa928274314537cf588126b95710633b2f92807fe94079a5d3519d4c0800
  Stored in directory: /tmp/pip-ephem-wheel-cache-nrg80wpt/wheels/64/69/41/6f820cf1d7488a0381a2059f66ec9f8f23116f7c67d18f3d8d
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
def sent_to_words(sentences):
  yield(gensim.utils.simple_preprocess(str(sentences).encode('utf-8'), deacc=True)) 

In [ ]:
def bi_tri_grams(poem_words):
  # Build the bigram and trigram models
  bigram = gensim.models.Phrases(poem_words, min_count=5, threshold=100) # higher threshold fewer phrases.
  trigram = gensim.models.Phrases(bigram[poem_words], threshold=100)  

  # Faster way to get a sentence clubbed as a trigram/bigram
  bigram_mod = gensim.models.phrases.Phraser(bigram)
  trigram_mod = gensim.models.phrases.Phraser(trigram)

  # See trigram example
  return bigram,trigram,bigram_mod,trigram_mod

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts,bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts,bigram_mod,trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
def get_clean_data(data_words,bigram_mod,trigram_mod):
  # Remove Stop Words
  data_words_nostops = remove_stopwords(data_words)

  # Form Bigrams
  data_words_bigrams = make_bigrams(data_words_nostops,bigram_mod)

  # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
  # python3 -m spacy download en
  nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

  # Do lemmatization keeping only noun, adj, vb, adv
  data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

  return data_lemmatized

In [ ]:
def create_corpus(data_lemmatized):
  # Create Dictionary
  id2word = corpora.Dictionary(data_lemmatized)

  # Create Corpus
  texts = data_lemmatized

  # Term Document Frequency
  corpus = [id2word.doc2bow(text) for text in texts]

  # View
  print(corpus[:1])
  return id2word,corpus

In [ ]:
def build_LDA_model(corpus,id2word):
  # Build LDA model
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=5, 
                                            random_state=200,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)
  return lda_model

In [ ]:
def get_lda_model_topics_topwords_print_top_topics(corpus,numtopics,id2word,random_state):
    lda_model = gensim.models.LdaMulticore(corpus, num_topics=numtopics, id2word=id2word, passes=3, workers=2, random_state=random_state)
    lda_all_topics=lda_model.show_topics(num_topics=numtopics, num_words=10,formatted=False)
    lda_topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in lda_all_topics]

    #Below Code Prints Topics and Words
    for topic,words in lda_topics_words:
        print(str(topic)+ "::"+ str(words))
    return lda_model,lda_topics_words

## **Poet 1**

In [ ]:
weldon_kees_df=pd.read_json('/content/weldon_kees.json')

In [ ]:
weldon_kees_text=weldon_kees_df.poem.str.cat(sep=', ')

In [ ]:
import re
weldon_kees_text=re.sub('[^A-Za-z0-9 ]+', '', weldon_kees_text)

In [ ]:
weldon_kees_words = list(sent_to_words(weldon_kees_text))
print(weldon_kees_words[:1])

[['prurient', 'tapirs', 'gamboled', 'on', 'our', 'lawns', 'but', 'that', 'was', 'quite', 'some', 'time', 'ago', 'now', 'one', 'is', 'accosted', 'by', 'asthmatic', 'bulldogs', 'sluggish', 'in', 'the', 'hedges', 'ruminant', 'moving', 'through', 'ivy', 'in', 'the', 'park', 'near', 'drying', 'waterfalls', 'we', 'open', 'every', 'gate', 'but', 'that', 'grave', 'shellwhite', 'unicorn', 'is', 'gone', 'the', 'path', 'is', 'strewn', 'with', 'papers', 'to', 'the', 'street', 'numbers', 'that', 'once', 'were', 'various', 'regarded', 'us', 'were', 'thought', 'significant', 'significant', 'enough', 'to', 'bring', 'reporters', 'to', 'the', 'scene', 'but', 'now', 'the', 'bell', 'strikes', 'one', 'strikes', 'one', 'strikes', 'onemonotonous', 'and', 'tired', 'or', 'clicks', 'like', 'sad', 'valise', 'note', 'to', 'be', 'left', 'on', 'the', 'table', 'this', 'ghost', 'of', 'yours', 'padding', 'about', 'the', 'upper', 'halls', 'given', 'to', 'frightwigs', 'burbage', 'might', 'have', 'worn', 'moaning', 'in',

In [ ]:
bigram1,trigram1,bigram_mod1,trigram_mod1 = bi_tri_grams(weldon_kees_words)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
weldon_kees_lemmatized = get_clean_data(weldon_kees_words,bigram_mod1,trigram_mod1)
print(weldon_kees_lemmatized[:1])

[['gambol', 'lawn', 'quite', 'time', 'ago', 'accost', 'asthmatic', 'bulldogs', 'sluggish', 'hedge', 'ruminant', 'move', 'park', 'dry', 'waterfall', 'open', 'gate', 'grave', 'shellwhite', 'unicorn', 'go', 'path', 'strew', 'paper', 'street', 'number', 'various', 'regard', 'think', 'significant', 'significant', 'enough', 'bring', 'reporter', 'strike', 'strike', 'strike', 'onemonotonous', 'tired', 'click', 'sad', 'valise', 'note', 'leave', 'table', 'ghost', 'pad', 'upper', 'hall', 'give', 'frightwig', 'burbage', 'worn', 'moaning', 'doorway', 'jump', 'maid', 'convince', 'even', 'life', 'frighten', 'enough', 'poor', 'pallid', 'counterpart', 'fuddle', 'role', 'inexcusable', 'go', 'haunt', 'house', 'girl', 'infect', 'man', 'bear', 'company', 'far', 'other', 'change', 'take', 'grave', 'edge', 'year', 'ago', 'dozen', 'hell', 'burn', 'purgatory', 'make', 'give', 'noon', 'ruin', 'traveler', 'sad', 'indolent', 'wear', 'fulfill', 'despise', 'rousseaus', 'prediction', 'perhaps', 'wait', 'bear', 'spot

In [ ]:
id2word1, corpus1 = create_corpus(weldon_kees_lemmatized)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 3), (6, 1), (7, 2), (8, 2), (9, 1), (10, 3), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 4), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 3), (41, 1), (42, 2), (43, 1), (44, 2), (45, 2), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 4), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 2), (60, 2), (61, 1), (62, 1), (63, 1), (64, 3), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 2), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 2), (101, 1), (102, 2), (103, 1), (104, 10), (105, 1), (106, 1), (107, 1), (108, 1), (109, 3), (110, 1

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word1[id], freq) for id, freq in cp] for cp in corpus1[:1]]

[[('accost', 1),
  ('actual', 1),
  ('add', 1),
  ('afternoon', 1),
  ('age', 1),
  ('ago', 3),
  ('agony', 1),
  ('ahead', 2),
  ('air', 2),
  ('ajar', 1),
  ('alone', 3),
  ('altar', 1),
  ('always', 1),
  ('amazingly', 1),
  ('ambiguous', 1),
  ('ancestor', 1),
  ('angel', 1),
  ('apoplectic', 1),
  ('appear', 1),
  ('arise', 1),
  ('arrange', 1),
  ('artist', 1),
  ('asthmatic', 1),
  ('asylum', 1),
  ('attack', 1),
  ('attic', 1),
  ('authority', 1),
  ('await', 1),
  ('awake', 1),
  ('baby', 1),
  ('back', 4),
  ('bad', 2),
  ('badly', 1),
  ('bank', 1),
  ('bar', 1),
  ('bark', 1),
  ('barricade', 1),
  ('baseball', 1),
  ('basement', 1),
  ('bastard', 1),
  ('bathing', 3),
  ('be', 1),
  ('bear', 2),
  ('beard', 1),
  ('beauty', 2),
  ('become', 2),
  ('bed', 1),
  ('beggar', 1),
  ('begin', 2),
  ('beside', 1),
  ('bird', 1),
  ('bite', 1),
  ('black', 4),
  ('bland', 1),
  ('blank', 1),
  ('bleach', 1),
  ('blind', 1),
  ('block', 1),
  ('blood', 1),
  ('blow', 2),
  ('blue',

In [ ]:
lda_model1 = build_LDA_model(corpus1,id2word1)

In [ ]:
# Print the Keyword in the 5 topics
pprint(lda_model1.print_topics())
doc_lda1 = lda_model1[corpus1]

[(0,
  '0.002*"come" + 0.002*"go" + 0.002*"year" + 0.002*"old" + 0.002*"eye" + '
  '0.002*"dead" + 0.002*"leave" + 0.002*"say" + 0.002*"end" + 0.002*"move"'),
 (1,
  '0.002*"come" + 0.002*"go" + 0.002*"old" + 0.002*"say" + 0.002*"leave" + '
  '0.002*"long" + 0.002*"year" + 0.002*"end" + 0.002*"eye" + 0.002*"die"'),
 (2,
  '0.002*"go" + 0.002*"come" + 0.002*"year" + 0.002*"say" + 0.002*"old" + '
  '0.002*"know" + 0.002*"eye" + 0.002*"dead" + 0.002*"leave" + 0.002*"end"'),
 (3,
  '0.002*"go" + 0.002*"come" + 0.002*"move" + 0.002*"year" + 0.002*"old" + '
  '0.002*"eye" + 0.002*"leave" + 0.002*"say" + 0.002*"long" + 0.002*"know"'),
 (4,
  '0.009*"go" + 0.009*"come" + 0.008*"year" + 0.007*"old" + 0.007*"leave" + '
  '0.007*"say" + 0.006*"end" + 0.006*"move" + 0.006*"know" + 0.006*"dead"')]


In [ ]:
lda_model_new1, lda_headline_topic_words1 = get_lda_model_topics_topwords_print_top_topics(
    corpus1, 5 ,id2word1,200)

0::['come', 'go', 'year', 'old', 'dead', 'leave', 'eye', 'end', 'move', 'say']
1::['come', 'go', 'old', 'say', 'long', 'leave', 'year', 'end', 'eye', 'die']
2::['go', 'come', 'year', 'say', 'old', 'know', 'eye', 'dead', 'leave', 'end']
3::['go', 'come', 'move', 'year', 'old', 'eye', 'leave', 'say', 'long', 'know']
4::['go', 'come', 'year', 'old', 'leave', 'say', 'end', 'dead', 'move', 'know']


## **Poet 2**

In [ ]:
alexander_pope_df=pd.read_json('/content/alexander_pope.json')

In [ ]:
alexander_pope_text=alexander_pope_df.poem.str.cat(sep=', ')

In [ ]:
import re
alexander_pope_text=re.sub('[^A-Za-z0-9 ]+', '', alexander_pope_text)

In [ ]:
alexander_pope_words = list(sent_to_words(alexander_pope_text))
print(alexander_pope_words[:1])

[['she', 'said', 'the', 'pitying', 'audience', 'melt', 'in', 'tears', 'but', 'fate', 'and', 'jove', 'had', 'stoppd', 'the', 'barons', 'ears', 'in', 'vain', 'thalestris', 'with', 'reproach', 'assails', 'for', 'who', 'can', 'move', 'when', 'fair', 'belinda', 'fails', 'not', 'half', 'so', 'fixd', 'the', 'trojan', 'could', 'remain', 'while', 'anna', 'beggd', 'and', 'dido', 'ragd', 'in', 'vain', 'then', 'grave', 'clarissa', 'graceful', 'wavd', 'her', 'fan', 'silence', 'ensud', 'and', 'thus', 'the', 'nymph', 'began', 'say', 'why', 'are', 'beauties', 'praisd', 'and', 'honourd', 'most', 'the', 'wise', 'mans', 'passion', 'and', 'the', 'vain', 'mans', 'toast', 'why', 'deckd', 'with', 'all', 'that', 'land', 'and', 'sea', 'afford', 'why', 'angels', 'calld', 'and', 'angellike', 'adord', 'why', 'round', 'our', 'coaches', 'crowd', 'the', 'whiteglovd', 'beaux', 'why', 'bows', 'the', 'sidebox', 'from', 'its', 'inmost', 'rows', 'how', 'vain', 'are', 'all', 'these', 'glories', 'all', 'our', 'pains', 'unl

In [ ]:
bigram2,trigram2,bigram_mod2,trigram_mod2 = bi_tri_grams(alexander_pope_words)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
alexander_pope_lemmatized = get_clean_data(alexander_pope_words,bigram_mod2,trigram_mod2)
print(alexander_pope_lemmatized[:1])

[['say', 'pity', 'audience', 'fate', 'baron', 'ear', 'vain', 'reproach', 'assail', 'move', 'fair', 'fail', 'half', 'fixd', 'trojan', 'remain', 'silence', 'ensud', 'thus', 'nymph', 'begin', 'say', 'beauty', 'passion', 'coach', 'crowd', 'row', 'vain', 'glory', 'pain', 'good', 'sense', 'preserve', 'beauty', 'gain', 'man', 'say', 'first', 'virtue', 'face', 'dance', 'night', 'old', 'age', 'away', 'scorn', 'housewife', 'care', 'produce', 'learn', 'earthly', 'thing', 'patch', 'become', 'saint', 'sure', 'sin', 'paint', 'ala', 'beauty', 'decay', 'lock', 'turn', 'paint', 'paint', 'fade', 'scorn', 'man', 'die', 'maid', 'remain', 'well', 'keep', 'good', 'humour', 'still', 'whateer', 'lose', 'trust', 'dear', 'good', 'humour', 'prevail', 'air', 'flight', 'scream', 'scold', 'fail', 'beauty', 'vain', 'pretty', 'eye', 'roll', 'charm', 'sight', 'merit', 'win', 'soul', 'speak', 'prude', 'arm', 'arm', 'fierce', 'cry', 'swift', 'lightning', 'combat', 'fly', 'side', 'party', 'begin', 'attack', 'fan', 'silk'

In [ ]:
id2word2, corpus2 = create_corpus(alexander_pope_lemmatized)

[[(0, 1), (1, 1), (2, 1), (3, 6), (4, 2), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 2), (11, 8), (12, 2), (13, 1), (14, 3), (15, 1), (16, 1), (17, 1), (18, 4), (19, 2), (20, 2), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 4), (32, 1), (33, 2), (34, 1), (35, 13), (36, 1), (37, 2), (38, 2), (39, 1), (40, 1), (41, 11), (42, 2), (43, 4), (44, 1), (45, 1), (46, 1), (47, 12), (48, 6), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 2), (58, 19), (59, 1), (60, 1), (61, 4), (62, 2), (63, 1), (64, 1), (65, 3), (66, 1), (67, 1), (68, 4), (69, 1), (70, 1), (71, 2), (72, 2), (73, 1), (74, 10), (75, 2), (76, 4), (77, 2), (78, 2), (79, 3), (80, 3), (81, 2), (82, 2), (83, 1), (84, 1), (85, 10), (86, 1), (87, 5), (88, 1), (89, 3), (90, 6), (91, 1), (92, 2), (93, 2), (94, 1), (95, 1), (96, 1), (97, 1), (98, 4), (99, 1), (100, 9), (101, 1), (102, 1), (103, 22), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 2), (

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word1[id], freq) for id, freq in cp] for cp in corpus1[:1]]

[[('accost', 1),
  ('actual', 1),
  ('add', 1),
  ('afternoon', 1),
  ('age', 1),
  ('ago', 3),
  ('agony', 1),
  ('ahead', 2),
  ('air', 2),
  ('ajar', 1),
  ('alone', 3),
  ('altar', 1),
  ('always', 1),
  ('amazingly', 1),
  ('ambiguous', 1),
  ('ancestor', 1),
  ('angel', 1),
  ('apoplectic', 1),
  ('appear', 1),
  ('arise', 1),
  ('arrange', 1),
  ('artist', 1),
  ('asthmatic', 1),
  ('asylum', 1),
  ('attack', 1),
  ('attic', 1),
  ('authority', 1),
  ('await', 1),
  ('awake', 1),
  ('baby', 1),
  ('back', 4),
  ('bad', 2),
  ('badly', 1),
  ('bank', 1),
  ('bar', 1),
  ('bark', 1),
  ('barricade', 1),
  ('baseball', 1),
  ('basement', 1),
  ('bastard', 1),
  ('bathing', 3),
  ('be', 1),
  ('bear', 2),
  ('beard', 1),
  ('beauty', 2),
  ('become', 2),
  ('bed', 1),
  ('beggar', 1),
  ('begin', 2),
  ('beside', 1),
  ('bird', 1),
  ('bite', 1),
  ('black', 4),
  ('bland', 1),
  ('blank', 1),
  ('bleach', 1),
  ('blind', 1),
  ('block', 1),
  ('blood', 1),
  ('blow', 2),
  ('blue',

In [ ]:
lda_model2 = build_LDA_model(corpus2,id2word2)

In [ ]:
# Print the Keyword in the 5 topics
pprint(lda_model2.print_topics())
doc_lda2 = lda_model2[corpus2]

[(0,
  '0.001*"still" + 0.001*"friend" + 0.001*"let" + 0.001*"make" + 0.000*"wit" + '
  '0.000*"see" + 0.000*"fool" + 0.000*"man" + 0.000*"passion" + 0.000*"sense"'),
 (1,
  '0.008*"still" + 0.006*"man" + 0.006*"make" + 0.006*"wit" + 0.005*"see" + '
  '0.005*"friend" + 0.005*"let" + 0.005*"know" + 0.005*"good" + 0.005*"eye"'),
 (2,
  '0.001*"still" + 0.001*"man" + 0.001*"make" + 0.001*"see" + 0.000*"good" + '
  '0.000*"let" + 0.000*"eye" + 0.000*"wit" + 0.000*"friend" + 0.000*"know"'),
 (3,
  '0.001*"still" + 0.000*"make" + 0.000*"see" + 0.000*"nature" + 0.000*"eye" + '
  '0.000*"wit" + 0.000*"let" + 0.000*"man" + 0.000*"friend" + 0.000*"evry"'),
 (4,
  '0.001*"still" + 0.001*"man" + 0.000*"let" + 0.000*"friend" + 0.000*"make" + '
  '0.000*"fool" + 0.000*"good" + 0.000*"think" + 0.000*"know" + 0.000*"wit"')]


In [ ]:
lda_model_new2, lda_headline_topic_words2 = get_lda_model_topics_topwords_print_top_topics(
    corpus2, 5 ,id2word2,200)

0::['still', 'friend', 'let', 'make', 'see', 'wit', 'man', 'eye', 'fool', 'nature']
1::['still', 'man', 'make', 'wit', 'see', 'friend', 'let', 'know', 'good', 'eye']
2::['still', 'man', 'make', 'see', 'good', 'let', 'eye', 'wit', 'know', 'friend']
3::['still', 'make', 'see', 'nature', 'eye', 'wit', 'let', 'man', 'friend', 'evry']
4::['still', 'man', 'let', 'friend', 'make', 'fool', 'good', 'think', 'know', 'wit']


# Name the topics:

In [ ]:
def build_word2Vec_model(data_lemmatized,min_count,size):
  model_w2vec = gensim.models.Word2Vec(data_lemmatized, size=size, window=10, min_count=min_count, workers=10)
  model_w2vec.train(data_lemmatized,total_examples=len(data_lemmatized),epochs=10)
  return model_w2vec

In [ ]:
def closest_word(a_word_tuple):
    closest = max(a_word_tuple,key=lambda item:item[1])
    return closest

In [ ]:
def find_commontopicname(lda_headline_topic_words,model_w2vec): 
  for topic,words in lda_headline_topic_words:
    mis_model = model_w2vec.wv.most_similar (positive=words)
    print(f"closest word to {str(words)} is {closest_word(mis_model)}")

## **Poet 1**

In [ ]:

model_w2vec1 = build_word2Vec_model(weldon_kees_lemmatized,1,6)

In [ ]:
find_commontopicname(lda_headline_topic_words1,model_w2vec1)

closest word to ['come', 'go', 'year', 'old', 'dead', 'leave', 'eye', 'end', 'move', 'say'] is ('corner', 0.9366803169250488)
closest word to ['come', 'go', 'old', 'say', 'long', 'leave', 'year', 'end', 'eye', 'die'] is ('high', 0.9441709518432617)
closest word to ['go', 'come', 'year', 'say', 'old', 'know', 'eye', 'dead', 'leave', 'end'] is ('corner', 0.9423036575317383)
closest word to ['go', 'come', 'move', 'year', 'old', 'eye', 'leave', 'say', 'long', 'know'] is ('corner', 0.9824224710464478)
closest word to ['go', 'come', 'year', 'old', 'leave', 'say', 'end', 'dead', 'move', 'know'] is ('first', 0.9520684480667114)


## **Poet 2**

In [ ]:

model_w2vec2 = build_word2Vec_model(alexander_pope_lemmatized,1,6)

In [ ]:
find_commontopicname(lda_headline_topic_words2,model_w2vec2)

closest word to ['still', 'friend', 'let', 'make', 'see', 'wit', 'man', 'eye', 'fool', 'nature'] is ('virtue', 0.9999810457229614)
closest word to ['still', 'man', 'make', 'wit', 'see', 'friend', 'let', 'know', 'good', 'eye'] is ('virtue', 0.9999588131904602)
closest word to ['still', 'man', 'make', 'see', 'good', 'let', 'eye', 'wit', 'know', 'friend'] is ('virtue', 0.9999587535858154)
closest word to ['still', 'make', 'see', 'nature', 'eye', 'wit', 'let', 'man', 'friend', 'evry'] is ('virtue', 0.9999775886535645)
closest word to ['still', 'man', 'let', 'friend', 'make', 'fool', 'good', 'think', 'know', 'wit'] is ('fair', 0.9999516010284424)


# Model Perplexity and Coherence Score

In [ ]:
def perplexity_coherence_calc(lda_model,corpus,data_lemmatized,id2word):
  # Compute Perplexity
  print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

  # Compute Coherence Score
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  print('\nCoherence Score: ', coherence_lda)


## **Poet 1**

In [ ]:
perplexity_coherence_calc(lda_model1,corpus1,weldon_kees_lemmatized,id2word1)


Perplexity:  -7.17765452876305

Coherence Score:  0.21126773131892707


## **Poet 2**

In [ ]:
perplexity_coherence_calc(lda_model2,corpus2,alexander_pope_lemmatized,id2word2)


Perplexity:  -7.832265450864021

Coherence Score:  0.23583355966441516


***REFERENCES***

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

https://radimrehurek.com/gensim/models/ldamodel.html 

https://github.com/aarsanjani/nlp-poets

